In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [3]:
data_yelp = pd.read_csv('/content/yelp_labelled.txt', sep='\t',header=None)

In [4]:
data_yelp.head()


,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [5]:
columan_name = ['Review', 'Sentiment']
data_yelp.columns = columan_name

In [6]:
data_yelp.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [7]:
data_yelp.shape

(1000, 2)

In [8]:
data_amazon = pd.read_csv('/content/amazon_cells_labelled.txt',sep='\t',header=None)
data_amazon.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [9]:
data_amazon.columns = columan_name

In [10]:
data_amazon.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [11]:
data_imdb = pd.read_csv('/content/imdb_labelled.txt',sep='\t',header=None)
data_imdb.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [12]:
data_imdb.columns = columan_name

In [13]:
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [14]:
data_imdb.shape

(748, 2)

In [19]:
data = pd.concat([data_yelp, data_amazon, data_imdb], ignore_index=True)

In [20]:
data.shape

(2748, 2)

In [23]:
data['Sentiment'].value_counts()
# 1386 positive reviews
# 1362 Negative reviews

,count
Sentiment,
1,1386
0,1362


In [24]:
data.isnull().sum()


,0
Review,0
Sentiment,0


In [25]:
x = data['Review']
y = data['Sentiment']

# Create function to clean the data

In [26]:
import string

In [27]:
punct = string.punctuation

In [28]:
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [29]:
from spacy.lang.en.stop_words import STOP_WORDS

In [30]:
stopwords = list(STOP_WORDS)

In [31]:
def text_data_cleaning(sentence):
  doc = nlp(sentence)

  tokens = [] # list of tokens
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)

  cleaned_tokens = []
  for token in tokens:
    if token not in stopwords and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens

In [32]:
text_data_cleaning("Hello all, It's a beautiful day outside there!")


['hello', 'beautiful', 'day', 'outside']

## Vectorization Feature Engineering (TF-IDF)

In [33]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [34]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)

In [35]:
classifier = LinearSVC()

## Splitting the dataset into the Train and Test set

In [36]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [37]:
x_train.shape, x_test.shape

((2198,), (550,))

In [38]:
x_train.head()

,Review
2572,"An Italian reviewer called this ""a small, grea..."
526,And it was way to expensive.
1509,"As an earlier review noted, plug in this charg..."
144,Nice blanket of moz over top but i feel like t...
2483,"The film gives meaning to the phrase, ""Never i..."


## Fit the x_train and y_train

In [39]:
clf = Pipeline([('tfidf',tfidf), ('clf',classifier)])
# it will first do vectorization of tfidf and then it will do classification

In [40]:
clf.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x7c0701757760>)),
                ('clf', LinearSVC())])

# Predict the Test set results

In [41]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [42]:
y_pred = clf.predict(x_test)

In [43]:
confusion_matrix(y_test, y_pred)

array([[201,  78],
       [ 50, 221]])

In [44]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.80      0.72      0.76       279
           1       0.74      0.82      0.78       271

    accuracy                           0.77       550
   macro avg       0.77      0.77      0.77       550
weighted avg       0.77      0.77      0.77       550



In [45]:
accuracy_score(y_test, y_pred)

0.7672727272727272

In [46]:
clf.predict(["Wow, I am learning Natural Language Processing in fun fashion!"])
# output is 1, that means review is positive

array([1])

In [47]:
clf.predict(["It's hard to learn new things!"])
# output is 0, that means review is Negative

array([0])